# Tech Store Sales Analysis**Created by:** Ranjith kumar Ramasamy**Purpose:** Education Purpose---

## Importing Necessary Libraries

In [ ]:
import osimport pandas as pdimport matplotlib.pyplot as plt

#### Merging the data from each month into on file

In [ ]:
path = r"C:\Users\parri\Downloads\SalesData"files = [file for file in os.listdir(path) if not file.startswith('.')] # Ignore hidden filesall_months_data = pd.DataFrame()for file in files:    current_data = pd.read_csv(path+"/"+file)    all_months_data = pd.concat([all_months_data, current_data])    all_months_data.to_csv("all_data.csv", index=False)

#### Reading in updated dataframe

In [ ]:
all_data = pd.read_csv("all_data.csv")all_data.head()

## Cleaning up the data

# Finding and dropping rows with NaN

In [ ]:
nan_df = all_data[all_data.isna().any(axis=1)]display(nan_df.head())all_data = all_data.dropna(how='all')all_data.head()

# Checking if the NaN were succesffully dropped

In [ ]:
nan_df = all_data[all_data.isna().any(axis=1)]display(nan_df.head())

# Removing text in order date column

In [ ]:
all_data = all_data[all_data['Order Date'].str[0:2]!='Or']

# Converting columns to the correct type

In [ ]:
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])all_data['Price Each'] = pd.to_numeric(all_data['Price Each'])

#### Augmenting data with additional columns

# Adding month column

In [ ]:
all_data['Month'] = all_data['Order Date'].str[0:2]all_data['Month'] = all_data['Month'].astype('int32')all_data.head()

# Adding city column

In [ ]:
def get_city(address):    return address.split(",")[1].strip(" ")def get_state(address):    return address.split(",")[2].split(" ")[1]all_data['City'] = all_data['Purchase Address'].apply(lambda x: f"{get_city(x)}  ({get_state(x)})")all_data.head()

## Data Exploration!### What was the best month for sales? How much was earned that month?

# Adding sales column 

In [ ]:
all_data['Sales'] = all_data['Quantity Ordered'].astype('int') * all_data['Price Each'].astype('float')all_data.head()

In [ ]:
all_data.groupby(['Month']).sum()

# Visualizing the data for Sales per Month

In [ ]:
months = range(1,13)print(months)plt.bar(months,all_data.groupby(['Month']).sum()['Sales'])plt.xticks(months)plt.title('Sales per Month')plt.ylabel('Sales (USD$)')plt.xlabel('Month')plt.show()

###### The best month for sales was December where they made a total of $4,613,443.34.

### What city sold the most product?

In [ ]:
all_data.groupby(['City']).sum()

# Visualizing the data for Sales per city

In [ ]:
keys = [city for city, df in all_data.groupby(['City'])]plt.bar(keys,all_data.groupby(['City']).sum()['Sales'])plt.title('Sales by City')plt.ylabel('Sales (USD$)')plt.xlabel('City')plt.xticks(keys, rotation='vertical', size=8)plt.show()

###### The city that sold the most products was San Francisco (CA).

###  What time should we display advertisements to maximize likelihood of customer's buying product?

# Adding time columns

In [ ]:
all_data['Hour'] = pd.to_datetime(all_data['Order Date']).dt.hourall_data['Minute'] = pd.to_datetime(all_data['Order Date']).dt.minuteall_data['Count'] = 1all_data.head()

# Using the order time from customers' orders to plot a graph

In [ ]:
keys = [pair for pair, df in all_data.groupby(['Hour'])]plt.plot(keys, all_data.groupby(['Hour']).count()['Count'])plt.xticks(keys)plt.title('Total Orders vs Time of Order')plt.ylabel('Total Number of Orders')plt.xlabel('Order Time (Hour)')plt.grid()plt.show()

#### The best time for ads to be aired would be 11AM-12PM or around 7PM.

### What product sold the most? Why do you think it sold the most?

In [ ]:
product_group = all_data.groupby('Product')quantity_ordered = product_group.sum()['Quantity Ordered']keys = [pair for pair, df in product_group]

In [ ]:
plt.bar(keys, quantity_ordered)plt.xticks(keys, rotation='vertical', size=10)plt.title('Total Number of Products Sold')plt.ylabel('Quantity Ordered')plt.xlabel('Product')plt.show()

#### The product sold the most were AAA Batteries. This may be due to a lot of electronics using them as a power source and its cost in comparison to the other items.

# Comparing cost of items to the quantity sold.

In [ ]:
prices = all_data.groupby('Product').mean()['Price Each']

In [ ]:
fig, ax1 = plt.subplots()ax2 = ax1.twinx()ax1.bar(keys, quantity_ordered, color='g')ax2.plot(keys, prices, color='b')ax1.set_xlabel('Product Name')ax1.set_ylabel('Quantity Ordered', color='g')ax2.set_ylabel('Price (USD$)', color='b')ax1.set_xticklabels(keys, rotation='vertical', size=8)fig.show()